# LSTM Return Prediction (2-day horizon)

This notebook mirrors the **MLP** notebook structure, but replaces the predictor with an **LSTM**.

- We build rolling windows from close prices (same `make_feature_windows` pipeline).
- Each **sample = one (window, asset)** pair.
- Input = the past sequence of returns (a 1D sequence), reshaped to `(timesteps, 1)`.
- Target = next-period return (configured so that **1 period = 2 trading days** via `days_per_week=2`).

The LSTM hyperparameter ranges and the “best” configuration are taken from the Ma et al. (2021) paper (Table 2 / Section 3.2).  


In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# project utilities (same as your MLP notebook)
from asset_data_module import read_close_prices_all_merged
from features import make_feature_windows

tf.keras.utils.set_random_seed(42)


In [77]:
# ----------------------------
# 1) Load data + build rolling windows
# ----------------------------
markets = ['dow30']
# markets = ['commodities']   # <-- change as needed

start_date, end_date = "2022-01-01", "2025-11-28"

_, close_df = read_close_prices_all_merged(markets, after_date=start_date)
close_df = close_df.loc[:end_date]

# IMPORTANT:
# days_per_week=2 means "1 period" corresponds to 2 trading days in your window builder.
# So horizon=1 => predicting the return over the next 2 trading days.
rolling = make_feature_windows(
    close_prices=close_df,
    lookback=60,
    horizon=1,
    days_per_week=2
)

close_df.shape, len(rolling)


((981, 30), 430)

In [78]:
# ----------------------------
# 2) Build panel dataset: (window, asset) -> (sequence, target)
# ----------------------------
def panel_from_windows(windows, x_key="past_weekly_returns", y_key="y_ret"):
    X_list, y_list = [], []
    meta_rows = []

    for w_idx, w in enumerate(windows):
        if x_key == 'past_weekly_returns':
            X_df = w[x_key].T          # assets x n_lookback (DataFrame)
        elif x_key == 'X_feat':
            X_df = w[x_key]            # assets x n_features (DataFrame)
        else:
            raise ValueError(f"Unknown x_key: {x_key}")

        y_ser = w[y_key]
        if not isinstance(y_ser, pd.Series):
            y_ser = pd.Series(y_ser, index=X_df.index)

        assets = X_df.index.intersection(y_ser.index)
        Xw = X_df.loc[assets].to_numpy(dtype=np.float32)
        yw = y_ser.loc[assets].to_numpy(dtype=np.float32)

        mask = np.isfinite(Xw).all(axis=1) & np.isfinite(yw)
        Xw, yw = Xw[mask], yw[mask]
        assets_kept = assets.to_numpy()[mask]

        X_list.append(Xw)
        y_list.append(yw)

        t0, t1 = w.get("t0", None), w.get("t1", None)
        for a in assets_kept:
            meta_rows.append((w_idx, a, t0, t1))

    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    meta = pd.DataFrame(meta_rows, columns=["window_idx", "asset", "t0", "t1"])
    return X, y, meta

# sequences: shape (samples, n_lookback)
X_seq_raw, y_raw, meta = panel_from_windows(rolling, x_key="past_weekly_returns", y_key="y_ret")

X_seq_raw.shape, y_raw.shape, meta.head()


((12900, 60),
 (12900,),
    window_idx       asset         t0         t1
 0           0  dow30:AAPL 2022-06-24 2022-06-28
 1           0  dow30:AMGN 2022-06-24 2022-06-28
 2           0   dow30:AXP 2022-06-24 2022-06-28
 3           0    dow30:BA 2022-06-24 2022-06-28
 4           0   dow30:CAT 2022-06-24 2022-06-28)

In [79]:
# ----------------------------
# 3) Train/val/test split by time (window index) + scaling
# ----------------------------
W = meta["window_idx"].nunique()
split_w = int(0.8 * W)

train_mask = (meta["window_idx"] < split_w).values
test_mask  = (meta["window_idx"] >= split_w).values

X_train_raw, y_train_raw = X_seq_raw[train_mask], y_raw[train_mask]
X_test_raw,  y_test_raw  = X_seq_raw[test_mask],  y_raw[test_mask]

# validation: last 10% of training windows
val_w = max(int(0.1 * split_w), 1)
val_start = split_w - val_w
val_mask = ((meta["window_idx"] >= val_start) & (meta["window_idx"] < split_w)).values

X_val_raw, y_val_raw = X_seq_raw[val_mask], y_raw[val_mask]

# remove val from train
train_mask2 = train_mask & (~val_mask)
X_train_raw, y_train_raw = X_seq_raw[train_mask2], y_raw[train_mask2]

print("Train/Val/Test shapes (raw):",
      X_train_raw.shape, X_val_raw.shape, X_test_raw.shape,
      y_train_raw.shape, y_val_raw.shape, y_test_raw.shape)

# --- scale X (fit on TRAIN only) ---
# For sequences, fit scaler on all training timesteps pooled together.
scaler_x = StandardScaler()
scaler_x.fit(X_train_raw.reshape(-1, 1))

def scale_seq(X):
    Xs = scaler_x.transform(X.reshape(-1, 1)).reshape(X.shape)
    return Xs.astype(np.float32)

X_train = scale_seq(X_train_raw)
X_val   = scale_seq(X_val_raw)
X_test  = scale_seq(X_test_raw)

# reshape to (samples, timesteps, features=1) for LSTM
X_train = X_train[..., None]
X_val   = X_val[..., None]
X_test  = X_test[..., None]

# --- scale y (optional but usually helps) ---
y_mean = y_train_raw.mean()
y_std  = y_train_raw.std() + 1e-8

def scale_y(y):
    return ((y - y_mean) / y_std).astype(np.float32)

def unscale_y(y_s):
    return (y_s * y_std + y_mean).astype(np.float32)

y_train = y_train_raw
y_val   = y_val_raw
y_test  = y_test_raw.astype(np.float32)

# y_train = scale_y(y_train_raw)
# y_val   = scale_y(y_val_raw)
# y_test  = y_test_raw.astype(np.float32)

X_train.shape, y_train.shape


Train/Val/Test shapes (raw): (9300, 60) (1020, 60) (2580, 60) (9300,) (1020,) (2580,)


((9300, 60, 1), (9300,))

In [ ]:
# ----------------------------
# 4) LSTM model (paper-style)
# ----------------------------
def build_lstm(
    timesteps,
    units=5,
    n_layers=4,
    dropout=0.4,
    recurrent_dropout=0.3,
    activation="relu",
    lr=0.01,
    optimizer_name="RMSprop",
    loss="mae",
):
    inp = keras.Input(shape=(timesteps, 1))
    x = inp

    for i in range(n_layers):
        return_seq = (i < n_layers - 1)
        x = layers.LSTM(
            units,
            activation=activation,
            dropout=dropout,
            recurrent_dropout=recurrent_dropout,
            return_sequences=return_seq,
        )(x)

    out = layers.Dense(1, activation="linear")(x)
    model = keras.Model(inp, out)

    opt_name = optimizer_name.lower()
    if opt_name == "rmsprop":
        opt = keras.optimizers.RMSprop(learning_rate=lr)
    elif opt_name == "adam":
        opt = keras.optimizers.Adam(learning_rate=lr)
    elif opt_name == "sgd":
        opt = keras.optimizers.SGD(learning_rate=lr)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.MeanAbsoluteError(name="mae")])
    return model

# paper's reported "best" LSTM config:
# 4 layers, 5 nodes each, lr=0.01, batch=100, dropout=0.4, rec_dropout=0.3, optimizer=RMSprop, loss=MAE
timesteps = X_train.shape[1]
# model = build_lstm(
#     timesteps=timesteps,
#     units=5,
#     n_layers=4,
#     dropout=0.4,
#     recurrent_dropout=0.3,
#     activation="relu",
#     lr=0.01,
#     # optimizer_name="RMSprop",
#     optimizer_name="adam",
#     loss="mae",
# )

model = build_lstm(
    timesteps=timesteps,
    units=5,
    n_layers=2,
    dropout=0.2,
    recurrent_dropout=0.2,
    activation="relu",
    # activation="tanh",
    lr=0.01,
    # optimizer_name="RMSprop",
    optimizer_name="adam",
    loss="mae",
)

model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 60, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 60, 5)          │           140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 5)              │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 366 (1.43 KB)

 Trainable params: 366 (1.43 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ----------------------------
# 5) Train
# ----------------------------
# epochs = 200
epochs = 20
batch_size = 100
patience = 0  # paper lists "patient=0" (we interpret this as "no early stopping")

callbacks = []
if patience and patience > 0:
    callbacks.append(
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True
        )
    )

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    callbacks=callbacks
)


Epoch 1/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0199 - mae: 0.0199 - val_loss: 0.0117 - val_mae: 0.0117
Epoch 2/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0117 - mae: 0.0117 - val_loss: 0.0117 - val_mae: 0.0117
Epoch 3/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0113 - mae: 0.0113 - val_loss: 0.0117 - val_mae: 0.0117
Epoch 4/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0111 - mae: 0.0111 - val_loss: 0.0116 - val_mae: 0.0116
Epoch 5/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0111 - mae: 0.0111 - val_loss: 0.0116 - val_mae: 0.0116
Epoch 6/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0111 - mae: 0.0111 - val_loss: 0.0116 - val_mae: 0.0116
Epoch 7/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0110 - mae: 0.0110 - val_loss: 0.0116 - val_mae: 0.0116
Epoch 8/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0110 - mae: 0.0110 - val_loss: 0.0116 - val_mae: 0.0116
Epoch 9/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.011

In [ ]:
# ----------------------------
# 6) Evaluate vs baseline (sample mean)
# ----------------------------
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# LSTM predictions (scaled -> unscaled)
y_pred_s = model.predict(X_test, batch_size=1024, verbose=0).squeeze()
y_pred = unscale_y(y_pred_s)

# baseline: sample mean of past sequence (unscaled)
y_pred_sm = X_test_raw.mean(axis=1).astype(np.float32)

def dir_acc(y_true, y_hat):
    return float((np.sign(y_true) == np.sign(y_hat)).mean())

metrics_lstm = {
    "MSE": mean_squared_error(y_test, y_pred),
    "MAE": mean_absolute_error(y_test, y_pred),
    "R2":  r2_score(y_test, y_pred),
    "DirAcc": dir_acc(y_test, y_pred),
}

metrics_sm = {
    "MSE": mean_squared_error(y_test, y_pred_sm),
    "MAE": mean_absolute_error(y_test, y_pred_sm),
    "R2":  r2_score(y_test, y_pred_sm),
    "DirAcc": dir_acc(y_test, y_pred_sm),
}

pd.DataFrame({"LSTM": metrics_lstm, "SampleMean": metrics_sm})


,LSTM,SampleMean
MSE,0.000466,0.000488
MAE,0.013221,0.013677
R2,-0.004991,-0.053229
DirAcc,0.545349,0.482946


In [ ]:
# ----------------------------
# 7) (Optional) Small grid search scaffold (paper lists large ranges in Table 2)
# ----------------------------
from itertools import product

grid = {
    "units": [5, 10, 15],
    "n_layers": [1, 2, 4],
    "lr": [1e-3, 1e-2],
    "batch_size": [50, 100],
    "dropout": [0.2, 0.4],
    "recurrent_dropout": [0.2, 0.3],
    "optimizer_name": ["RMSprop", "Adam"],
    "activation": ["relu"],
    "loss": ["mae"],
}

def train_eval_one(cfg, epochs=100, patience=5):
    tf.keras.utils.set_random_seed(42)
    tf.keras.backend.clear_session()

    m = build_lstm(
        timesteps=X_train.shape[1],
        units=cfg["units"],
        n_layers=cfg["n_layers"],
        dropout=cfg["dropout"],
        recurrent_dropout=cfg["recurrent_dropout"],
        activation=cfg["activation"],
        lr=cfg["lr"],
        optimizer_name=cfg["optimizer_name"],
        loss=cfg["loss"],
    )

    callbacks = []
    if patience and patience > 0:
        callbacks.append(keras.callbacks.EarlyStopping(monitor="val_loss", patience=patience, restore_best_weights=True))

    m.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=cfg["batch_size"],
        verbose=0,
        callbacks=callbacks
    )

    y_pred_s = m.predict(X_val, batch_size=1024, verbose=0).squeeze()
    y_pred = unscale_y(y_pred_s)
    y_true = y_val_raw.astype(np.float32)

    return {
        **cfg,
        "val_MAE": mean_absolute_error(y_true, y_pred),
        "val_MSE": mean_squared_error(y_true, y_pred),
        "val_DirAcc": dir_acc(y_true, y_pred),
    }

# NOTE: this can be slow if you expand the grid.
keys = list(grid.keys())
configs = [dict(zip(keys, vals)) for vals in product(*[grid[k] for k in keys])]

results = []
for i, cfg in enumerate(configs[:10]):  # <-- limit to first N configs for a quick run
    res = train_eval_one(cfg, epochs=80, patience=10)
    results.append(res)
    print(i, res["val_MAE"], cfg)

df_grid = pd.DataFrame(results).sort_values("val_MAE")
df_grid.head(10)


0 0.011637250892817974 {'units': 5, 'n_layers': 1, 'lr': 0.001, 'batch_size': 50, 'dropout': 0.2, 'recurrent_dropout': 0.2, 'optimizer_name': 'RMSprop', 'activation': 'relu', 'loss': 'mae'}
1 0.01163478847593069 {'units': 5, 'n_layers': 1, 'lr': 0.001, 'batch_size': 50, 'dropout': 0.2, 'recurrent_dropout': 0.2, 'optimizer_name': 'Adam', 'activation': 'relu', 'loss': 'mae'}
2 0.011637943796813488 {'units': 5, 'n_layers': 1, 'lr': 0.001, 'batch_size': 50, 'dropout': 0.2, 'recurrent_dropout': 0.3, 'optimizer_name': 'RMSprop', 'activation': 'relu', 'loss': 'mae'}
3 0.011640357784926891 {'units': 5, 'n_layers': 1, 'lr': 0.001, 'batch_size': 50, 'dropout': 0.2, 'recurrent_dropout': 0.3, 'optimizer_name': 'Adam', 'activation': 'relu', 'loss': 'mae'}
4 0.011633374728262424 {'units': 5, 'n_layers': 1, 'lr': 0.001, 'batch_size': 50, 'dropout': 0.4, 'recurrent_dropout': 0.2, 'optimizer_name': 'RMSprop', 'activation': 'relu', 'loss': 'mae'}
5 0.011636395007371902 {'units': 5, 'n_layers': 1, 'lr': 

,units,n_layers,lr,batch_size,dropout,recurrent_dropout,optimizer_name,activation,loss,val_MAE,val_MSE,val_DirAcc
4,5,1,0.001,50,0.4,0.2,RMSprop,relu,mae,0.011633,0.00029,0.503922
6,5,1,0.001,50,0.4,0.3,RMSprop,relu,mae,0.011634,0.00029,0.503922
1,5,1,0.001,50,0.2,0.2,Adam,relu,mae,0.011635,0.00029,0.503922
7,5,1,0.001,50,0.4,0.3,Adam,relu,mae,0.011635,0.00029,0.503922
5,5,1,0.001,50,0.4,0.2,Adam,relu,mae,0.011636,0.00029,0.503922
0,5,1,0.001,50,0.2,0.2,RMSprop,relu,mae,0.011637,0.00029,0.503922
8,5,1,0.001,100,0.2,0.2,RMSprop,relu,mae,0.011638,0.00029,0.503922
2,5,1,0.001,50,0.2,0.3,RMSprop,relu,mae,0.011638,0.00029,0.503922
3,5,1,0.001,50,0.2,0.3,Adam,relu,mae,0.011640,0.00029,0.503922
9,5,1,0.001,100,0.2,0.2,Adam,relu,mae,0.011642,0.00029,0.503922


## Notes vs the paper

- Table 2 lists the hyperparameter search ranges for LSTM: hidden nodes, hidden layers, learning rate, patience, batch size, dropout, recurrent dropout, loss, optimizer.  
- The paper's reported “best” LSTM is: **4 layers**, **5 nodes**, **lr=0.01**, **batch=100**, **dropout=0.4**, **recurrent dropout=0.3**, optimizer **RMSprop**, loss **MAE**, and they use **relu** as activation.  

If you want to match their “patient=0” behavior exactly, keep early stopping disabled (as above).  
If you want a more stable training loop, use a positive early-stopping patience and monitor `val_loss`.
